In [2]:
import os

In [3]:
%pwd

'e:\\SOWMYA\\MachineLearning\\End-to-end-ML-Project-Implementation\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'e:\\SOWMYA\\MachineLearning\\End-to-end-ML-Project-Implementation'

In [6]:
# entity: return type of a function

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.param = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request 
import zipfile 
from mlProject import logger 
from mlProject.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config 

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(F"{filename} download! with following info: \n{headers}")
        else:
            logger.ingo(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-13 20:11:31,898: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-13 20:11:31,899: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-13 20:11:31,900: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-13 20:11:31,901: INFO: common: created directory at: artifacts]
[2023-10-13 20:11:31,902: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-13 20:11:32,939: INFO: 1153713965: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 21952
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "a372375009998659389e6518c1b818d180d1ff599f860d8bafbb5c198018e058"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A866:5EC06:81F98:D21D9:6529571C
Accept-Ranges: bytes
Date: Fri, 13 Oc